In [1]:
import pandas as pd
 
input_book = pd.ExcelFile('/home/brain/Downloads/心理検査DB_差分入り.xlsx')
result_lh_df = pd.read_table("/home/brain/Downloads/lh.ilf.All.csv")
result_rh_df = pd.read_table("/home/brain/Downloads/rh.ilf.All.csv")

#sheet_namesメソッドでExcelブック内の各シートの名前をリストで取得できる
input_sheet_name = input_book.sheet_names

/home/brain/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  after removing the cwd from sys.path.
/home/brain/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """


In [2]:
#DataFrameとして一つ目のsheetを読込
fu_sheet_df = input_book.parse(input_sheet_name[0])
fu_sheet_df=fu_sheet_df[fu_sheet_df[fu_sheet_df.columns[0]].str.contains("",na=False)]

#読み込んだシート名の確認
print("Sheet name:", input_sheet_name[0])

fu_sheet_df=fu_sheet_df.iloc[:,0:24]
fu_sheet_df=fu_sheet_df.drop(fu_sheet_df.columns[[21,22]],axis=1)
#読み込んだシートの先頭5行を表示
print(fu_sheet_df.shape)#head(5))

Sheet name: follow_without29
(51, 22)


In [3]:
#DataFrameとして3つ目のsheetを読込（グラフはsheetとしてカウントされない？）
bl_sheet_df = input_book.parse(input_sheet_name[1])
bl_sheet_df=bl_sheet_df[bl_sheet_df[bl_sheet_df.columns[0]].str.contains("",na=False)]

#読み込んだシート名の確認
print("Sheet name:", input_sheet_name[1])

bl_sheet_df=bl_sheet_df.drop(bl_sheet_df.columns[[21,23,27,28]],axis=1)
#読み込んだシートの先頭5行を表示
print(bl_sheet_df.shape)#head(5))

Sheet name: baseline_without29
(51, 29)


In [4]:
bl_fu_sheet_df=pd.merge(bl_sheet_df, fu_sheet_df, left_on=bl_sheet_df.columns[0], right_on=fu_sheet_df.columns[0], how="inner", suffixes = ["_bl", "_fu"])
bl_fu_sheet_df.columns=bl_fu_sheet_df.columns.str.replace(" ","_")
bl_fu_sheet_df.columns=bl_fu_sheet_df.columns.str.replace("\n","_")
print(bl_fu_sheet_df.shape)#head(5))

(51, 50)


In [5]:
result_bilateral_df=pd.merge(result_lh_df, result_rh_df, left_on=result_lh_df.columns[0], right_on=result_rh_df.columns[0], how="inner", suffixes = ["_lh", "_rh"])
print(result_bilateral_df.shape)#head(5))

(103, 38)


In [6]:
bl_bilateral_df=result_bilateral_df[result_bilateral_df[result_bilateral_df.columns[0]].str.endswith('_1')]
fu_bilateral_df=result_bilateral_df[result_bilateral_df[result_bilateral_df.columns[0]].str.endswith('_2')]
print(bl_bilateral_df.shape)#head(5))
print(fu_bilateral_df.shape)#head(5))

(51, 38)
(52, 38)


In [7]:
bl_bilateral_df["研究ID"]=bl_bilateral_df[bl_bilateral_df.columns[0]].str[0:6]
fu_bilateral_df["研究ID"]=fu_bilateral_df[fu_bilateral_df.columns[0]].str[0:6]
bl_fu_bilateral_df=pd.merge(bl_bilateral_df.drop(columns=bl_bilateral_df.columns[0]), fu_bilateral_df.drop(columns=fu_bilateral_df.columns[0]), on="研究ID", how="inner", suffixes = ["_bl", "_fu"]).sort_values(by="研究ID")
print(bl_fu_bilateral_df.shape)#head(5))

(51, 75)


/home/brain/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/brain/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [8]:
df_result=pd.merge(bl_fu_sheet_df, bl_fu_bilateral_df, left_on=bl_fu_sheet_df.columns[0], right_on="研究ID", how="inner").sort_values(by="研究ID")
print(df_result.shape)
print(df_result)

(50, 124)
      研究ID     検査日_bl  年齢_bl  性別_1：男性_２：女性_bl  TMT-A_（秒）_bl  TMT-B_（秒）_bl  \
0   U28002 2016-08-10   64.0              2.0          88.0          88.0   
1   U28003 2016-08-02   63.0              2.0          97.0         122.0   
2   U28004 2016-08-03   62.0              2.0          64.0          79.0   
3   U28005 2016-08-19   58.0              2.0          44.0          86.0   
4   U28008 2016-08-23   67.0              1.0          98.0         222.0   
5   U28009 2016-08-01   63.0              1.0          89.0         166.0   
6   U28010 2016-08-24   60.0              2.0          66.0          84.0   
7   U28011 2016-08-24   66.0              2.0         101.0         119.0   
8   U28013 2016-08-10   59.0              1.0          63.0          88.0   
9   U28014 2016-08-10   57.0              2.0          85.0          86.0   
10  U28015 2016-09-01   63.0              2.0         106.0          95.0   
11  U28017 2016-09-06   64.0              2.0          56.0       

In [9]:
df_result.to_csv('/home/brain/Downloads/df_result.csv')